#  Tutorial of Simple Seq2seq with Teacher forcing

### 1) Read Data

reference : https://www.tensorflow.org/tutorials/text/nmt_with_attention?hl=zh_cn

In [0]:
import json
import io
import re
import os
import time
import random
import numpy as np
import unicodedata
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
# %tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
    raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Tensorflow version 2.2.0-rc4
Running on TPU  ['10.80.83.210:8470']
INFO:tensorflow:Initializing the TPU system: grpc://10.80.83.210:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.80.83.210:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [3]:
path_to_zip = tf.keras.utils.get_file(
    'spa-eng.zip', origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
    extract=True)

path_to_file = os.path.dirname(path_to_zip)+"/spa-eng/spa.txt"

2646016/2638744 [==============================] - 0s 0us/step


### 2) Text PreProcessing

#### (1) Text Cleaning

In [0]:
# 将 unicode 文件转换为 ascii
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

In [0]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"}

In [0]:
def preprocess_sentence(sentence):
    
    sentence = unicode_to_ascii(sentence.lower().strip())
    
    sentence = " ".join([contractions[word] if word in contractions else word for word in sentence.split(' ')])
    
    # 在单词与跟在其后的标点符号之间插入一个空格
    # 例如： "he is a boy." => "he is a boy ."
    # reference：https://stackoverflosentence.com/questions/3645931/python-padding-punctuation-sentenceith-sentencehite-spaces-keeping-punctuation
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)

    # 除了 (a-z, A-Z, ".", "?", "!", ",")，将所有字符替换为空格
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)

    sentence = sentence.rstrip().strip()

    # 给句子加上开始和结束标记
    # 以便模型知道何时开始和结束预测
    sentence = '<start> ' + sentence + ' <end>'
    return sentence

In [7]:
en_sentence = u"May I borrow this book?"
sp_sentence = u"¿Puedo tomar prestado este libro?"
print(preprocess_sentence(en_sentence))
print(preprocess_sentence(sp_sentence).encode('utf-8'))

<start> may i borrow this book ? <end>
b'<start> \xc2\xbf puedo tomar prestado este libro ? <end>'


In [0]:
# 1. 去除重音符号
# 2. 清理句子
# 3. 返回这样格式的单词对：[ENGLISH, SPANISH]
def create_dataset(path, num_examples):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')[:-1]

    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]

    return zip(*word_pairs)

In [0]:
en, sp = create_dataset(path_to_file, None)

In [10]:
print(en[29000])
print(sp[29000])

<start> tadpoles become frogs . <end>
<start> los renacuajos se convierten en ranas . <end>


#### (2) sentence tokenizing

In [0]:
def max_length(tensor):
    return max(len(t) for t in tensor)

def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    
    lang_tokenizer.fit_on_texts(lang)
    
    tensor = lang_tokenizer.texts_to_sequences(lang)

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,padding='post')
    
    return tensor, lang_tokenizer

In [0]:
def load_dataset(path, num_examples=None):
    # 创建清理过的输入输出对
    targ_lang, inp_lang = create_dataset(path, num_examples)

    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

    return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [0]:
# 尝试实验不同大小的数据集
num_examples = 30000
input_tensor, output_tensor, input_tokenizer, output_tokenizer = load_dataset(path_to_file, num_examples)

# 计算目标张量的最大长度 （max_length）
max_length_targ, max_length_inp = max_length(output_tensor), max_length(input_tensor)

In [0]:
max_length_output, max_length_input = max_length(output_tensor), max_length(input_tensor)

In [15]:
# 采用 80 - 20 的比例切分训练集和验证集
input_tensor_train, input_tensor_val, output_tensor_train, output_tensor_val = train_test_split(input_tensor, output_tensor, test_size=0.2)

# 显示长度
print(len(input_tensor_train), len(output_tensor_train), len(input_tensor_val), len(output_tensor_val))

24000 24000 6000 6000


In [0]:
def convert(lang, tensor):
    for t in tensor:
        if t!=0:
            print ("%d ----> %s" % (t, lang.index_word[t]))

In [17]:
input_tensor_train[1000]

array([  1, 350,  14,  16, 623,   3,   2,   0,   0,   0,   0,   0,   0,
         0,   0,   0], dtype=int32)

In [18]:
print ("Input Language; index to word mapping")
convert(input_tokenizer, input_tensor_train[7000])
print ()
print ("Target Language; index to word mapping")
convert(output_tokenizer, output_tensor_train[7000])

Input Language; index to word mapping
1 ----> <start>
25 ----> ella
42 ----> tiene
21 ----> una
409 ----> cara
1060 ----> bonita
3 ----> .
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
26 ----> she
50 ----> has
9 ----> a
213 ----> pretty
439 ----> face
3 ----> .
2 ----> <end>


#### (3) DataSet Creating

In [0]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64

In [0]:
dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, output_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [21]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 16]), TensorShape([64, 11]))

In [22]:
example_input_batch.shape

TensorShape([64, 16])

In [0]:
# example_input_batch

### 2) Encoder

##### 1) Model

In [0]:
class Encoder(tf.keras.Model):
    def __init__(self,vocab_size,embedding_dim,encode_units):
        super(Encoder,self).__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.encode_units = encode_units
        self.embedding = tf.keras.layers.Embedding(self.vocab_size,self.embedding_dim)
        self.gru = tf.keras.layers.GRU(units=self.encode_units,return_sequences=True,return_state=True)
        
    
    def call(self,x):
        encoder_embedding = self.embedding(x)
        encode_output,encode_hidden_state = self.gru(encoder_embedding)
        
        return encode_output,encode_hidden_state
        

##### 2) parameter

In [0]:
INPUT_VOCAB_SIZE = len(input_tokenizer.word_index)+1
ENCODER_EMBEDDING_SIZE = 256
ENCODER_UNIT = 1024

In [26]:
INPUT_VOCAB_SIZE

9414

In [0]:
encoder = Encoder(INPUT_VOCAB_SIZE,ENCODER_EMBEDDING_SIZE,ENCODER_UNIT)

In [0]:
encode_output,encode_hidden_state = encoder(example_input_batch)

In [29]:
encode_hidden_state

<tf.Tensor: shape=(64, 1024), dtype=float32, numpy=
array([[-0.01884282,  0.00875622,  0.00219726, ...,  0.00774832,
        -0.00805246, -0.01802043],
       [-0.01860362,  0.00864717,  0.0023609 , ...,  0.00770958,
        -0.00788831, -0.01813345],
       [-0.01864812,  0.0086738 ,  0.00229547, ...,  0.00769006,
        -0.00799331, -0.01801563],
       ...,
       [-0.01842104,  0.00855171,  0.00256057, ...,  0.00771517,
        -0.00790944, -0.01803633],
       [-0.01875985,  0.00869879,  0.0022138 , ...,  0.00773742,
        -0.00802038, -0.01803016],
       [-0.01880912,  0.00874791,  0.00221335, ...,  0.00775205,
        -0.00801045, -0.01805654]], dtype=float32)>

### 3) Decoder

##### 1) Model

In [0]:
class Decoder(tf.keras.Model):
    def __init__(self,vocab_size,decode_unit,embedding_dim):
        super(Decoder,self).__init__()
        self.vocab_size = vocab_size
        self.decode_unit = decode_unit
        self.embedding_dim = embedding_dim
        
        ### structure
        self.gru =  tf.keras.layers.GRU(units=self.decode_unit,return_sequences=True,return_state=True)
        
        self.embeddding = tf.keras.layers.Embedding(self.vocab_size,self.embedding_dim)
        
        self.fc = tf.keras.layers.Dense(self.vocab_size)
        
        
        ### decode_input  【batch_size,word_index】
    def call(self,decode_input,encode_output):
        
        decode_input = self.embeddding(decode_input)
        
        shape = (decode_input.shape[0],encode_output.shape[1]-decode_input.shape[1],decode_input.shape[2])
        
        padding = tf.zeros(shape)
        
        decode_input = tf.concat([decode_input,padding],axis = 1)
        
        concat_vector = tf.concat([encode_output,decode_input,decode_input],axis = -1)
        
        decode_output,decode_hidden_state = self.gru(concat_vector)
        
        decode_output = tf.reduce_sum(decode_output,axis = 1)
        
        y = self.fc(decode_output)
        
        return y
        

##### 2) Parameter

In [0]:
### parameter
output_vocab_size = len(output_tokenizer.word_index)+1

DECODER_UNIT = ENCODER_UNIT

encode_embedding_dim = 256

In [32]:
output_vocab_size

4921

In [0]:
decoder = Decoder(output_vocab_size,DECODER_UNIT,encode_embedding_dim)

In [0]:
decode_input = example_target_batch[:, :1]
# decode_input = tf.convert_to_tensor([output_tokenizer.word_index['<start>']] * BATCH_SIZE)

In [0]:
# decode_input

In [0]:
predictions = decoder(decode_input,encode_output)

In [37]:
predictions

<tf.Tensor: shape=(64, 4921), dtype=float32, numpy=
array([[-0.07986051, -0.02821629, -0.0072703 , ..., -0.03879301,
        -0.02286101, -0.04484978],
       [-0.05277197, -0.00912803, -0.00510451, ..., -0.02388329,
        -0.00487622, -0.03686088],
       [-0.03598433, -0.01482207, -0.01312359, ..., -0.01404658,
        -0.02321418, -0.03029034],
       ...,
       [-0.02261956, -0.01800052, -0.01717209, ..., -0.01200012,
         0.01241633, -0.02250165],
       [-0.05983879, -0.02388211, -0.00662596, ..., -0.03637584,
        -0.0120089 , -0.06147018],
       [-0.07153626, -0.02314482, -0.00849418, ..., -0.03579164,
        -0.01180682, -0.05038257]], dtype=float32)>

### 4) Define Loss Function

In [0]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    
    loss_ *= mask

    return tf.reduce_mean(loss_)

### 5) Save Model

In [0]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

### 6）Train Function

In [0]:
@tf.function
def train_step(inp,tar):
    
    loss = 0
    
    with tf.GradientTape() as tape:
    
        encode_output,encode_hidden_state = encoder(inp)

        decode_input = tf.convert_to_tensor([output_tokenizer.word_index['<start>']] * BATCH_SIZE)

#         end_index = tf.cast(tf.argmin(tf.reduce_sum(targ,axis = 0)),tf.int32)

        for t in range(1,tar.shape[1]):
        
            decode_input = targ[:,:t]
        
            predictions = decoder(decode_input,encode_output)

            loss += loss_function(tar[:,t],predictions)
        
    
    batch_loss = (loss / int(targ.shape[1]))
    
    variables = encoder.trainable_variables + decoder.trainable_variables
    
    gradients = tape.gradient(loss,variables)
    
    
    optimizer.apply_gradients(zip(gradients, variables))
    
    return batch_loss
    
    

In [0]:
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE

In [43]:
EPOCHS = 20

for epoch in range(EPOCHS):
    start = time.time()

    total_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ)
        total_loss += batch_loss

        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                     batch,
                                                     batch_loss.numpy()))
    # 每 2 个周期（epoch），保存（检查点）一次模型
    # if (epoch + 1) % 2 == 0:
    #     checkpoint.save(file_prefix = checkpoint_prefix)
        
    print('Epoch {} Loss {:.4f}'.format(epoch + 1,total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 0.7033
Epoch 1 Batch 100 Loss 0.6861
Epoch 1 Batch 200 Loss 0.8124
Epoch 1 Batch 300 Loss 0.6923
Epoch 1 Loss 0.6809
Time taken for 1 epoch 255.28366231918335 sec

Epoch 2 Batch 0 Loss 0.4936
Epoch 2 Batch 100 Loss 0.6094
Epoch 2 Batch 200 Loss 0.6386
Epoch 2 Batch 300 Loss 0.4522
Epoch 2 Loss 0.4715
Time taken for 1 epoch 255.84842085838318 sec

Epoch 3 Batch 0 Loss 0.3371
Epoch 3 Batch 100 Loss 0.3008
Epoch 3 Batch 200 Loss 0.3419
Epoch 3 Batch 300 Loss 0.2415
Epoch 3 Loss 0.3591
Time taken for 1 epoch 255.48558402061462 sec

Epoch 4 Batch 0 Loss 0.2646
Epoch 4 Batch 100 Loss 0.2403
Epoch 4 Batch 200 Loss 0.2298
Epoch 4 Batch 300 Loss 0.2870
Epoch 4 Loss 0.2911
Time taken for 1 epoch 256.9820203781128 sec

Epoch 5 Batch 0 Loss 0.1636
Epoch 5 Batch 100 Loss 0.2076
Epoch 5 Batch 200 Loss 0.2050
Epoch 5 Batch 300 Loss 0.2465
Epoch 5 Loss 0.2384
Time taken for 1 epoch 256.2942724227905 sec

Epoch 6 Batch 0 Loss 0.1929
Epoch 6 Batch 100 Loss 0.1961
Epoch 6 Batch 200 L

In [44]:
# 恢复检查点目录 （checkpoint_dir） 中最新的检查点
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [0]:
def evaluate(sentence):
#     attention_plot = np.zeros((max_length_targ, max_length_inp))

    sentence = preprocess_sentence(sentence)

    inputs = [input_tokenizer.word_index[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                           maxlen=max_length_input,
                                                           padding='post')
    inputs = tf.convert_to_tensor(inputs)

    result = ''
    
#     encode_output,encode_hidden_state = encoder(inputs)
    enc_out, enc_hidden = encoder(inputs)

    dec_input = tf.expand_dims([output_tokenizer.word_index['<start>']],1)

    predicted_ids = [1]
    for t in range(max_length_targ):
        
        predictions = decoder(dec_input,enc_out)

        predicted_id = tf.argmax(predictions[0]).numpy()
        
        if output_tokenizer.index_word[predicted_id] == '<end>':
            return result, sentence
        else:
            if predicted_ids[-1]!=predicted_id:
                predicted_ids.append(predicted_id)

        result = ' '.join([output_tokenizer.index_word[predicted_id] for predicted_id in predicted_ids])

        # 预测的 ID 被输送回模型
        dec_input = tf.expand_dims(predicted_ids,0)
        
    
    return result, sentence

In [0]:
def translate(sentence):
    result, sentence = evaluate(sentence)

    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))

In [52]:
translate(u'no me gustan las.')

Input: <start> no me gustan las . <end>
Predicted translation: <start> i do not like them


In [57]:
translate(u'¿todavia estan en casa.')

Input: <start> ¿ todavia estan en casa . <end>
Predicted translation: <start> are you still


In [55]:
translate(u'trata de averiguarlo.')

Input: <start> trata de averiguarlo . <end>
Predicted translation: <start> try


In [67]:
sp[8800][8:-6]

'nos vemos a las cinco .'

In [68]:
translate(sp[8800][8:-6])

Input: <start> nos vemos a las cinco . <end>
Predicted translation: <start> see


In [69]:
en[8800]

'<start> see you at five . <end>'